# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [4]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 
from pyspark.sql.functions import udf
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
spark = SparkSession.builder.appName('data wrangling').getOrCreate()

In [5]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [6]:
user_log.show()

+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession| lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+---------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       Showaddywaddy|Logged In|  Kenneth|     M|          112| Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|          Lily Allen|Logged In|Elizabeth|     F|            7|    Chase|195.23873| free|Shreveport-Bossie...|   PUT

In [7]:
user_log.createOrReplaceTempView('user_log_table')

# Question 1

Which page did user id ""(empty string) NOT visit?

In [8]:
# TODO: write your code to answer question 1
spark.sql('''
            select distinct page
            from user_log_table
            where userId == ''
''').show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [15]:
# TODO: write your code to answer question 3
spark.sql('''
            select count(distinct userId)
            from user_log_table
            where gender = 'F'
''').show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [20]:
# TODO: write your code to answer question 4
spark.sql('''
            select Artist, count(Artist)
            from user_log_table
            group by Artist
            order by count(Artist) DESC
            limit 1
''').show()

+--------+-------------+
|  Artist|count(Artist)|
+--------+-------------+
|Coldplay|           83|
+--------+-------------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [71]:
# TODO: write your code to answer question 5
is_home = spark.sql('''select userId, page, ts,
                        case when page = 'Home' then 1 else 0 end as is_home
                        from user_log_table
''')
is_home.createOrReplaceTempView("is_home_table")

In [75]:

cumulative_sum = spark.sql("SELECT *, SUM(is_home) OVER \
    (PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period \
    FROM is_home_table")
cumulative_sum.createOrReplaceTempView("period_table")

In [78]:
avg = spark.sql('''
                    select avg(*)
                    from (SELECT COUNT(*) AS count_results FROM period_table \
                    GROUP BY userID, period, page 
                    HAVING page = 'NextSong') as counts
''').show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

